신경망 가중치 매개변수의 기울기는 수치 미분 사용해서 구함 (시간이 오래 걸림)

# 오차역전파법

수식 (기계학습), 계산 그래프

# 계산 그래프

계산 과정을 그래프로 나타낸 것

1. 계산 그래프 구성
2. 왼쪽에서 오른쪽으로 계산 진행 (순전파)

## 국소적 계산

국소적 계산 (자신과 직접 관계된 작은 범위)

각 노드는 자신과 관련한 계산 외에는 신경쓸 것이 없음

## 계산 그래프
장점
- 국소적 계산 (문제의 단순화)
- 중간 계산 결과를 모두 보관
- 순전파와 역전파를 통해 미분을 효율적으로 계산 가능

# 연쇄법칙

역전파는 국소적인 미분을 반대 반향으로 전달 (연쇄법칙에 따른 것)

## 역전파
신호 노드의 국소적 미분(순전파 때 계산의 미분)을 곱한 후 다음 노드로 전달

## 연쇄법칙
합성 함수: 여러 함수로 구성된 함수

합성 함수의 미분은 합성 함수를 구성하는 각 함수의 미분의 곱으로 나타낼 수 있음

# 역전파

## 덧셈 노드
덧셈 노드의 역전파는 1을 곱하기만 하고 입력된 값을 그대로 다음 노드로 보냄

## 곱셈 노드
입력 신호들을 서로 바꾼 값을 곱해서 보냄

# 계층 구현

## 곱셈 계층

forward(): 순전파
backward(): 역전파

In [1]:
class MulLayer:
    def __init__(self):
        self.x = None
        self.y = None

    def forward(self, x, y):
        self.x = x
        self.y = y
        out = x * y

        return out

    def backward(self, d_out):
        dx = d_out * self.y
        dy = d_out * self.x

        return dx, dy

In [2]:
# 순전파 구현
apple = 100
apple_num = 2
tax = 1.1

mul_apple_layer = MulLayer()
mul_tax_layer = MulLayer()

# forward
apple_price = mul_apple_layer.forward(apple, apple_num)
price = mul_tax_layer.forward(apple_price, tax)

print(price)

220.00000000000003


In [3]:
# 역전파 구현
d_price = 1
d_apple_price, d_tax = mul_tax_layer.backward(d_price)
d_apple, d_apple_num = mul_apple_layer.backward(d_apple_price)

print(d_apple, d_apple_num, d_tax)

2.2 110.00000000000001 200


## 덧셈 계층

In [4]:
class AddLayer:
    def __init__(self):
        pass

    def forward(self, x, y):
        out = x + y
        return out

    def backward(self, d_out):
        d_x = d_out * 1
        d_y = d_out * 1
        return d_x, d_y

In [5]:
# 순전파와 역전파
apple = 100
apple_num = 2
orange = 150
orange_num = 3
tax = 1.1

# 계층
mul_apple_layer = MulLayer()
mul_orange_layer = MulLayer()
add_apple_orange_layer = AddLayer()
mul_tax_layer = MulLayer()

# forward
apple_price = mul_apple_layer.forward(apple, apple_num)
orange_price = mul_orange_layer.forward(orange, orange_num)
all_price = add_apple_orange_layer.forward(apple_price, orange_price)
price = mul_tax_layer.forward(all_price, tax)

print(apple_price, orange_price, all_price, price)

200 450 650 715.0000000000001


In [6]:
d_price = 1
d_all_price, d_tax = mul_tax_layer.backward(d_price)
d_apple_price, d_orange_price = add_apple_orange_layer.backward(d_all_price)
d_orange, d_orange_num = mul_orange_layer.backward(d_orange_price)
d_apple, d_apple_num = mul_apple_layer.backward(d_apple_price)

print(d_apple_num, d_apple, d_orange, d_orange_num, d_tax)

110.00000000000001 2.2 3.3000000000000003 165.0 650


# 활성화 함수 계층 구현

## ReLU 계층
```
return x if x > 0 else 0
```
미분
```
return 1 if x > 0 else 0
```

In [19]:
class Relu:
    def __init__(self):
        self.mask = None

    def forward(self, x):
        self.mask = (x <= 0) # True, False로 구성된 numpy 배열
        out = x.copy()
        out[self.mask] = 0

        return out

    def backward(self, d_out):
        d_out[self.mask] = 0
        d_x = d_out

        return d_x

전기회로 스위치와 유사한 원리(흐르고 있으면 ON, 아니면 OFF)

## Sigmoid 계층

순전파: 입력 => x 를 -1과 곱함 => exp (지수 값 연산) => 1을 더함 => 1로 나눔 => 출력
역전파: 출력 => 제곱 후 마이너스 => 그대로 => exp (지수 값 연산) => -1 곱함 => 입력

계층 간소화
순전파: sigmoid(x)
역전파: y^2exp(-x) => y(1-y) # 순전파의 출력만으로 계산 가능!!

In [17]:
import numpy as np

class Sigmoid:
    def __init__(self):
        self.out = None

    def forward(self, x):
        self.out = 1 / (1 + np.exp(-x))

        return self.out

    def backward(self, d_out):
        d_x = d_out * (1.0 - self.out) * self.out

        return d_x

# Affine / Softmax 계층 구현

## Affine 계층
affine transformation(어파인 변환): 신경망의 순전파 때 수행하는 행렬의 곱
순전파: X * W + B = Y
역전파
 - X = Y * W^T
 - W = (X)^T * Y

행렬의 형상에 주의: 행렬 곱에서 대응하는 차원의 원소 수 일치 작업

## 배치용 Affine 계층
데이터 여러 개를 묶어서 순전파하는 경우

In [2]:
import numpy as np

X_dot_W = np.array([[0, 0, 0], [10, 10, 10]])
B = np.array([1, 2, 3])

In [3]:
X_dot_W + B

array([[ 1,  2,  3],
       [11, 12, 13]])

In [5]:
# 순전파의 편향 덧셈
dY = np.array([[1, 2, 3], [4, 5, 6]])

dY

array([[1, 2, 3],
       [4, 5, 6]])

In [12]:
dB = np.sum(dY, axis=0)

dB

array([5, 7, 9])

Affine 구현

In [20]:
class Affine:
    def __init__(self, W, b):
        self.W = W
        self.b = b
        self.x = None
        self.dW = None
        self.db = None

    def forward(self, x):
        self.x = x
        out = np.dot(x, self.W) + self.b

        return out

    def backward(self, d_out):
        dx = np.dot(d_out, self.W.T)
        self.dW = np.dot(self.x.T, d_out)
        self.db = np.sum(d_out, axis=0)

        return dx

## Softmax-with-Loss 계층

Softmax: 입력값을 정규화하여 출력

입력 이미지가 Affine 계층과 ReLU 계층을 통과하며 변환
마지막 Softmax 계층에 의해 입력이 정규화

Softmax에 교차 엔트로피 오차를 포함하여 구현

Softmax 계층: 입력을 정규화하여 출력
Cross Entropy Error 계층: Softmax 출력과 정답 레이블을 받아서 손실 L 출력

신경망의 출력이 정답 레이블과 가까워지도록 가중치 매개변수의 값을 조정
(효율적으로 앞 계층에 전달하기 위해 '소프트맥스 함수'의 손실 함수로 '교차 엔트로피 오차'를 사용 -> 차가 직접적으로 전달됨)

In [21]:
class SoftmaxWithLoss:
    def __init__(self):
        self.loss = None
        self.y = None
        self.t = None
    
    def forward(self, x, t):
        self.t = t
        self.y = softmax(x)
        self.loss = cross_entropy_error(self.y, self.t)
        return self.loss
    
    def backward(self, d_out=1):
        batch_size = self.t.shape[0]
        d_x = (self.y - self.t) / batch_size
        return d_x

# 오차역전파법 구현

## 신경망 학습의 전체 그림
미니배치 -> 기울기 산출 -> 매개변수 갱신 -> 반복

In [4]:
import sys, os
sys.path.append(os.pardir)

import numpy as np
from libs.layers import *
from collections import OrderedDict

class TwoLayerNet:
    def __init__(self, input_size, hidden_size, output_size, weight_init_std=0.01):
        # initialize
        self.params = {}
        self.params["W1"] = weight_init_std * np.random.randn(input_size, hidden_size)
        self.params["b1"] = np.zeros(hidden_size)
        self.params["W2"] = weight_init_std * np.random.randn(hidden_size, output_size)
        self.params["b2"] = np.zeros(output_size)
        
        # 계층 생성
        self.layers = OrderedDict() # 순서가 있는 dictionary
        self.layers["Affine1"] = Affine(self.params["W1"], self.params["b1"])
        self.layers["Relu1"] = Relu()
        self.layers["Affine2"] = Affine(self.params["W2"], self.params["b2"])
        
        self.lastLayer = SoftmaxWithLoss()
    
    def predict(self, x):
        for layer in self.layers.values():
            x = layer.forward(x)
            
        return x
    
    # x: input data, t: label
    def loss(self, x, t):
        y = self.predict(x)
        return self.lastLayer.forward(y, t)
    
    def accuracy(self, x, t):
        y = self.predict(x)
        y = np.argmax(y, axis=1)
        if t.ndim != 1 : t = np.argmax(t, axis=1)
        
        accuracy = np.sum(y == t) / float(x.shape[0])
        return accuracy
    
    def numerical_gradient(self, x, t):
        loss_W = lambda W: self.loss(x, t)
        
        grads = {}
        grads["W1"] = numerical_gradient(loss_W, self.params["W1"])
        grads["b1"] = numerical_gradient(loss_W, self.params["b1"])
        grads["W2"] = numerical_gradient(loss_W, self.params["W2"])
        grads["b2"] = numerical_gradient(loss_W, self.params["b2"])
        return grads
    
    def gradient(self, x, t):
        # 순전파
        self.loss(x, t)
        
        d_out = 1
        d_out = self.lastLayer.backward(d_out)
        
        layers = list(self.layers.values())
        layers.reverse()
        for layer in layers:
            d_out = layer.backward(d_out)
        
        grads = {}
        grads["W1"] = self.layers["Affine1"].dW
        grads["b1"] = self.layers["Affine1"].db
        grads["W2"] = self.layers["Affine2"].dW
        grads["b2"] = self.layers["Affine2"].db
        
        return grads

## 오차역전파법으로 구한 기울기 검증
기울기를 구하는 방법: 수치 미분 (느림), 해석적 수식 (오차역전파법, 매개변수가 많아도 효율적으로 계산 가능)
수치 미분은 정확히 구현했는지 확인하는 데에 사용

In [7]:
import sys, os
sys.path.append(os.pardir)

import numpy as np
from datasets.mnist import load_mnist
from libs.layers import TwoLayerNet

(X_train, y_train), (X_test, y_test) = load_mnist(normalize=True, one_hot_label=True)

network = TwoLayerNet(input_size=784, hidden_size=50, output_size=10)

X_batch = X_train[:3]
y_batch = y_train[:3]

grad_numerical = network.numerical_gradient(X_batch, y_batch)
grad_backprop = network.gradient(X_batch, y_batch)

for key in grad_numerical.keys():
    diff = np.average(np.abs(grad_backprop[key] - grad_numerical[key]))
    print(key + ": " + str(diff))

W1: 4.050495633426916e-10
b1: 2.6666336826686664e-09
W2: 6.024618149412891e-09
b2: 1.405530836010027e-07


## 오차역전파법 사용 학습 구현

In [1]:
import numpy as np
from datasets.mnist import load_mnist
from libs.layers import TwoLayerNet

(X_train, y_train), (X_test, y_test) = load_mnist(normalize=True, one_hot_label=True)
network = TwoLayerNet(input_size=784, hidden_size=50, output_size=10)

iters_num = 10000
train_size = X_train.shape[0]
batch_size = 100
learning_rate = 0.1

train_loss_list = []
train_acc_list = []
test_acc_list = []

iter_per_epoch = max(train_size / batch_size, 1)

for i in range(iters_num):
    batch_mask = np.random.choice(train_size, batch_size)
    X_batch = X_train[batch_mask]
    y_batch = y_train[batch_mask]
    
    # solve gradient
    grad = network.gradient(X_batch, y_batch)
    
    for key in ("W1", "b1", "W2", "b2"):
        network.params[key] -= learning_rate * grad[key]
    
    loss = network.loss(X_batch, y_batch)
    train_loss_list.append(loss)
    
    if i % iter_per_epoch == 0:
        train_acc = network.accuracy(X_train, y_train)
        test_acc = network.accuracy(X_test, y_test)
        train_acc_list.append(train_acc)
        test_acc_list.append(test_acc)
        print(train_acc, test_acc)

0.09708333333333333 0.0978
0.9025 0.9066
0.925 0.9243
0.9347666666666666 0.9337
0.9435666666666667 0.9423
0.94565 0.945
0.9552833333333334 0.9534
0.95905 0.9562
0.9626166666666667 0.9604
0.9653333333333334 0.9607
0.9684166666666667 0.9636
0.971 0.9669
0.9736166666666667 0.9673
0.9745166666666667 0.9684
0.9767666666666667 0.9694
0.9781166666666666 0.9701
0.9771166666666666 0.9696
